# Introduction

This notebook is meant as an introduction to random forests. Random forests are an ensemble method that can be used for supervised classification of data. After walking through the structure of a random forest algorith, we will apply the idea to a dataset of network intrusions to create an intrusion decection classifier. 

## Part 1: Building Intuition

In [87]:
## Import block
%matplotlib inline
%load_ext autoreload
%autoreload


import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import numpy as np
import pandas as pd

from random import seed

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.tree import export_graphviz
# from sklearn.externals.six import StringIO 
from IPython.display import Image 
from pydot import graph_from_dot_data
from six import StringIO

import random

from helperFunctions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Import Data
For the sake of understanding, we will first walk through creating a random forest with an easy to understand dataset of dog breeds, with the goal of classifying them as good or not good for novice owners. Once we have built the understanding, we will then apply the algorithm to a less user friendly dataset of network activity. 

The dog dataset has 3 features: "Easy to Train", "Kid Friendly", and "High Energy".

In [74]:
## Import Binary Dog Data 

dog_pd = pd.read_csv("lab16data-binary.csv", sep = ",", index_col = "Breed Name")

dog_pd.head()

,Easy To Train,Kid-Friendly,High-Energy,Good For Novice Owners
Breed Name,,,,
Afador,False,False,True,False
Affenhuahua,False,False,True,True
Affenpinscher,False,False,True,True
Afghan Hound,False,True,True,False
Airedale Terrier,True,True,True,False


### Ensemble Methods

These are methods in machine learning that use a group of methods to perform a task. First we run each method individually, then use some aggregate of their results to determine a final result. In the case of random forests, that means we create several trees to classify our data, then aggregate their classifications (perhaps by taking the most popular one) to get a final classification. It is common to use the majority to make this decision, but it is also possible to use a different voting system including weighting votes.

#### How to get different results?
Say we build several decision trees on the same dataset with the same classification purpose. We will get several copies of the same tree! We need to make each tree a little different by adding some element of randomness. What we can do is select a random subset of the features of the data. In this case since we only have 3 features, we can just build three trees, excluding one feature from the fit of each tree/

In [24]:
## Tree 1: Excluding "High-Energy"
build_decision_tree(dog_pd, "Good For Novice Owners", exclude_features=['High-Energy'])

>Splitting 391 data points on Kid-Friendly
>Produces 288 True data points and 103 False data points
>>Splitting 288 data points on Easy To Train
>>Produces 160 True data points and 128 False data points
>>Splitting 103 data points on None
>>No best next split.


In [25]:
## Tree 2: Excluding "Kid-Friendly"
build_decision_tree(dog_pd, "Good For Novice Owners", exclude_features=['Kid-Friendly'])

>Splitting 391 data points on Easy To Train
>Produces 198 True data points and 193 False data points
>>Splitting 198 data points on None
>>No best next split.
>>Splitting 193 data points on None
>>No best next split.


In [26]:
## Tree 3: Excluding "Easy To Train"
build_decision_tree(dog_pd, "Good For Novice Owners", exclude_features=['Easy To Train'])

>Splitting 391 data points on Kid-Friendly
>Produces 288 True data points and 103 False data points
>>Splitting 288 data points on None
>>No best next split.
>>Splitting 103 data points on None
>>No best next split.


Creating trees from different combinations of input variables gives us some idea of which variables are the most important to the classifications. For instance, 2 of the three trees we just built split on kid friendly first, which might indicate that kid-friendly is an important factor.

### Pruning Trees
Decision trees tend to overfit to their training data, creating giant trees with leaves at the bottom containing small subsets of the data.

This is a question of the bias-variance trade-off. We defined models which exhibit high _variance_ as models that if we were to change the data slightly or show the model new data, then the tree may change drastically. Models with high variance (ie. those that are overfit to the data) are said to be hard to generalize. They also might simply contain too many rules or decisions for assigning a label (or class) to a datapoint. In a sense, to avoid overfitting, we want nice compact trees where each node contributes more to the classification than the effort of adding that node to the tree.

One way to avoid overfitting with trees is to create **pruned** trees, where branches that do not contribute much to the tree are cut off. To do so we must first create the whole tree to determine which variables matter, then prune, rather than just creating short trees using the max_depth parameter.

### sklearn for Decision Trees

The sklearn module allows pruning of trees with the ccp_alpha parameter. Larger values result in more pruning. To experiment with this we need a larger dataset than the one we have used so far:



In [27]:
# import full dog data
dog_full_pd = pd.read_csv("lab16data.csv", sep = ",", index_col = "Breed Name")
dog_full_np = dog_full_pd.to_numpy(dtype = np.float16)

In [28]:
# view full dog data
dog_full_pd.head()

,Size,"Avg. Life Span, years",Wanderlust Potential,Adaptability,All Around Friendliness,Health And Grooming Needs,Physical Needs,Easy To Train,Kid-Friendly,High-Energy,Good For Novice Owners
Breed Name,,,,,,,,,,,
Afador,4,11,4,2.4,3.00,3.2,4.00,False,False,True,False
Affenhuahua,1,16,2,3.0,3.00,3.2,3.33,False,False,True,True
Affenpinscher,1,13,2,3.2,3.33,2.4,3.33,False,False,True,True
Afghan Hound,4,11,5,4.0,4.67,2.0,3.67,False,True,True,False
Airedale Terrier,3,12,4,2.2,4.00,2.4,4.33,True,True,True,False


In [29]:
# Split into the input variables and the target classes
in_dog_data = dog_full_np[:,:-1]
out_class = dog_full_np[:,-1]

# Get the variable names 
var_names = list(dog_full_pd.columns)[:-1]

In [30]:
# prepare three decision tree classifiers, with alphas {0.1, 0.01, 0.001}
dt1 = DecisionTreeClassifier(ccp_alpha = 0.001)
dt2 = DecisionTreeClassifier(ccp_alpha = 0.01)
dt3 = DecisionTreeClassifier(ccp_alpha = 0.1)

In [31]:
# train three decision tree classifiers
dt1.fit(in_dog_data, out_class)
dt2.fit(in_dog_data, out_class)
dt3.fit(in_dog_data, out_class)

DecisionTreeClassifier(ccp_alpha=0.1)

In [32]:
# graph dt1 (ccp_alpha = 0.001)
dot_data = StringIO()

export_graphviz(dt1, out_file=dot_data, feature_names=var_names)
(dt_vis, ) = graph_from_dot_data(dot_data.getvalue())

Image(dt_vis.create_png())

FileNotFoundError: [Errno 2] "dot" not found in path.

In [ ]:
# graph dt2 (ccp_alpha = 0.01)
dot_data = StringIO()

export_graphviz(dt2, out_file=dot_data, feature_names=var_names)
(dt_vis, ) = graph_from_dot_data(dot_data.getvalue())

Image(dt_vis.create_png())

In [ ]:
# graph dt3 (ccp_alpha = 0.1)
dot_data = StringIO()

export_graphviz(dt3, out_file=dot_data, feature_names=var_names)
(dt_vis, ) = graph_from_dot_data(dot_data.getvalue())

Image(dt_vis.create_png())

### Building a Random Forest
With this intuition about pruning and taking results from multiple trees, we can now move on to random forests.

A random forest classifies data based on a majority vote from a grove of pruned, random decision trees. We can start by creating 3 pruned decision trees with 4 randomly chosen features per tree.

In [ ]:
# build three trees with the same criteria
seed(2022)
dt1 = DecisionTreeClassifier(max_features = 4, ccp_alpha = 0.01)
dt2 = DecisionTreeClassifier(max_features = 4, ccp_alpha = 0.01)
dt3 = DecisionTreeClassifier(max_features = 4, ccp_alpha = 0.01)

In [ ]:
# train all three - due to randomness, will be different trees
dt1.fit(in_dog_data, out_class)
dt2.fit(in_dog_data, out_class)
dt3.fit(in_dog_data, out_class)

In [ ]:
# predict for the first ten dogs using the first classifier
dt1.predict(in_dog_data[:10,:])

# predict for the first ten dogs using the second classifier
dt2.predict(in_dog_data[:10,:])

# predict for the first ten dogs using the third classifier
dt3.predict(in_dog_data[:10,:])

The classifications of our three trees on this dog (where 1 means a dog that IS good for novice owners) were 1, 0,0. Voting by simple majority, this dog is not good for novice owners.

## Part 2: Random Forest Intrusion Detection System

Having seen how a random forest works, we can now use sklearns built in method for creating one. Since we already have an intution gained from working with this straightforward dog data set, we can now pivot to working with the network activity dataset. This data records TCP/IP connections in an LAN network, with variables including protocol, application accessed, size of packet, etc. We will build a random forest on this data, then test its performance as an intrusion dection system. 

In [46]:
# Import new dataset
data = pd.read_csv("Train_data.csv")
data_wrangle(data, ["protocol_type", "service", "flag", "class"])
data_np = np.array(data)
training_index = random.choices(range(len(data_np)), k = 4500)
training_np = data_np[training_index, :].astype('int')

testing_index = list(set(range(len(data_np)))-set(training_index))
testing_np = data_np[testing_index, :].astype('int')

/var/folders/f2/0nnwfhvd6rn_f569pqrzdsvw0000gn/T/ipykernel_60196/1279583825.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column][dataset[column] == var] = replacements[var]


In [110]:
print(training['class'])
#training_np[:5, :]

0        0
1        0
2        1
3        0
4        0
        ..
25187    1
25188    1
25189    1
25190    1
25191    1
Name: class, Length: 25192, dtype: object


In [67]:
grove = RandomForestClassifier(n_estimators=10, max_features=10, max_depth=10, random_state=0)

In [68]:
in_data = training_np[:, :-1]
out_class = training_np[:, -1]

In [69]:
# Fit our model to the data
grove.fit(in_data, out_class)

RandomForestClassifier(max_depth=10, max_features=10, n_estimators=10,
                       random_state=0)

In [70]:
predictions = grove.predict(testing_np[:, :-1])

print(classification_mse(testing_np[:, -1], predictions))

0.006219141663501709


In [99]:
# compare some different configurations of number of estimator trees
print(randomForestCV(training_np[:, :-1], training_np[:, -1], 10, n_estimators = 10, max_features = 10, max_depth = 10))
print(randomForestCV(training_np[:, :-1], training_np[:, -1], 10, n_estimators = 20, max_features = 10, max_depth = 10))
print(randomForestCV(training_np[:, :-1], training_np[:, -1], 10, n_estimators = 15, max_features = 10, max_depth = 10))


0.007333333333333333
0.0077777777777777776
0.006888888888888889


In [101]:
# different configurations of max_features
print(randomForestCV(training_np[:, :-1], training_np[:, -1], 10, n_estimators = 15, max_features = 5, max_depth = 10))
print(randomForestCV(training_np[:, :-1], training_np[:, -1], 10, n_estimators = 15, max_features = 12, max_depth = 10))
print(randomForestCV(training_np[:, :-1], training_np[:, -1], 10, n_estimators = 15, max_features = 8, max_depth = 10))

0.008222222222222223
0.008222222222222223
0.009555555555555557


In [106]:
print(randomForestCV(training_np[:, :-1], training_np[:, -1], 10, n_estimators = 15, max_features = 10, max_depth = 5))
print(randomForestCV(training_np[:, :-1], training_np[:, -1], 10, n_estimators = 15, max_features = 10, max_depth = 15))
print(randomForestCV(training_np[:, :-1], training_np[:, -1], 10, n_estimators = 15, max_features = 10, max_depth = 18))
print(randomForestCV(training_np[:, :-1], training_np[:, -1], 10, n_estimators = 15, max_features = 10, max_depth = 20))

0.017777777777777778
0.006222222222222222
0.006666666666666666
0.0064444444444444445


After some perhaps overly finnicky variations and cross validation to improve cross val performance by a tiny fraction, it seems that a random forest with 15 estimators, 10 random features, and a max depth of 15 will generally perform very well. Let's now train a model with those hyper parameters on the full dataset and test it with the test data

In [111]:
final_grove = RandomForestClassifier(n_estimators=15, max_features=10, max_depth=15, random_state=0)
final_grove.fit(training_np[:, :-1], training_np[:, -1])
final_predictions = final_grove.predict(testing_np[:, :-1])

print(classification_mse(testing_np[:, -1], final_predictions))
#out of curiosity, let's count percentages of false pos/false neg
# 1 = anomaly, 0 = normal
false_pos = 0
false_neg = 0
for i in range(len(final_predictions)):
    if final_predictions[i]!=testing_np[i, -1]:
        if final_predictions[i]> testing_np[i, -1]:
            false_pos = false_pos + 1
        else:
            false_neg = false_neg + 1
false_pos = false_pos/len(final_predictions)
false_neg = false_neg/len(final_predictions)


0.005222180022787694


In [112]:
print(false_pos)
print(false_neg)

0.0015191796429927839
0.003703000379794911


We have trained a random forest on network connections that can classify activity as normal or anomalous with a means squared error of about .005, which indicates a very accurate classifier. Of course, in a real-world implementation, flagging  ~.15% of normal network traffic as anomalous may waste valuable resources investigating harmless activity, and disregarding ~.4% of anomalous behaviour as normal may allow a damaging attack to go undetected, but it is also possible that a commercial intrusion detection system would have a more finely calibrated classifier trained on a larger dataset. This classifier serves to illustrate how those real-world IDS's can work.